### A8: Generative Models 生成式模型
- 在上个章节中，我们介绍了自动编码器及其变种。`自动编码机(AutoEncoders)`通过学习y=x的恒等变换实现在无监督训练自动提取特征。自动编码机的变种，包括`稀疏自动编码机（SparseAEs）`、`去噪自动编码机（DenoisingAEs）`，通过训练技巧提升了编码的稀疏性和鲁棒性。这一类常常被用于特征提取和降维任务。但是对于生成任务而言，他们并不合适，原因在于原始数据分布在编码器复杂非线性变换的投射下未必是规则的，这不利于我们在生成采样时令解码器得到有意义的结果。
- 变分自动编码机（VAE），借鉴了贝叶斯方法中的变分推断技术，在训练模型学习恒等变换的同时通过引入对编码空间的KL散度进行正则化，使得编码空间在编码过程中保持良好的规范性，有利于在编码空间的采样与插值。
- VAE也可以实现一种有监督的变种：条件变分自动编码机（ConditionalVAE，CVAE），可引入标签数据，实现有标签的生成任务。

VAE的原理：
- 首先，它在AutoEncoder的基础上，假定编码器的结果是一个高斯分布，用于近似后验条件分布P(z|x)，前向传播时对它的均值与方差进行预测。
- VAEs在训练时采用重参数化技巧，计算z=mu + std * eps, （eps从高斯分布上采样得到），使得梯度可以顺利回传至编码器。
- 使用重构误差（y与x的误差）和编码空间的KL散度损失同时对模型进行约束。

本实验工作内容如下：
- 以`CIFAR-10`和`MNIST`数据集为例，展示条件变分自动编码机（ConditionalVAE，CVAE）的训练和评估
- 提供了美丽的UI界面，用于展示潜在空间的连续性
- 复现了CVAE作为贝叶斯模型，在训练时遇到的“后验坍缩”问题

### 零、安装依赖

In [1]:
!pip install torch torchvision
!pip install numpy
!pip install matplotlib
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### 一、导入模块

In [2]:
import os
from enum import Enum

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [8]:
class ConditionalConvVAE(nn.Module):
    """
    条件全卷积VAE变分自动编码器
    """
    NAME = 'ConditionalConvVAE'
    
    def __init__(self, potential_dim, channels, num_classes=10):
        super(ConditionalConvVAE, self).__init__()
        self.potential_dim = potential_dim
        self.channels = channels
        
        # 对类别标签进行编码的线性层
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        
        output_shape = (1024, 4, 4)
        
        output_dim = output_shape[0] * output_shape[1] * output_shape[2]
        
        # 编码器
        self.encoder = nn.Sequential(
            nn.Conv2d(channels + num_classes, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 1024, kernel_size=3, stride=2, padding=1), # output: 1024 x 8 x 8
            nn.ReLU(),
            nn.Flatten(),
        )
        
        
        self.enc_mu = nn.Linear(output_dim, potential_dim)     # 均值
        self.enc_log_var = nn.Linear(output_dim, potential_dim) # 对数方差
        # 解码器
        self.decoder_fc = nn.Linear(potential_dim + num_classes, output_dim)
        
        self.decoder = nn.Sequential(
            nn.Unflatten(1, output_shape),
            nn.ConvTranspose2d(1024, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, channels, kernel_size=3, stride=2, padding=1, output_padding=1),   # output: channel x 28 x 28
            nn.Sigmoid()
        )

    def encode(self, x, labels):
        # 将标签嵌入到与图像相同的维度
        labels = self.label_embedding(labels).unsqueeze(2).unsqueeze(3)  # (128, 10, 1, 1)
        labels = labels.expand(labels.size(0), labels.size(1), x.size(2), x.size(3))
        
        # 将标签和图像连接起来
        x = torch.cat((x, labels), dim=1)
        
        # 传入编码器
        x = self.encoder(x)
        mu = self.enc_mu(x)
        log_var = self.enc_log_var(x)
        
        # 重参数化
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        z = mu + eps * std
        
        return z, mu, log_var

    def decode(self, z, labels):
        # 将标签嵌入并与潜在向量连接起来
        labels = self.label_embedding(labels)
        z = torch.cat((z, labels), dim=1)

        # 传入解码器
        x = self.decoder_fc(z)
        x = self.decoder(x)
        return x

    def forward(self, x, labels):
        z, mu, log_var = self.encode(x, labels)
        reconstructed_x = self.decode(z, labels)
        return reconstructed_x, mu, log_var


# 定义损失函数
def vae_loss(recon_x, x, mu, log_var):
    # 重构损失：通常使用二元交叉熵（BCE）损失
    MSE = nn.functional.mse_loss(recon_x.view(-1, 32*32), x.view(-1, 32*32), reduction='sum')
    # KL 散度损失：用于度量学到的潜在分布与标准正态分布之间的差异
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    loss = MSE + KLD
    return loss, np.array([loss.item(), MSE.item(), KLD.item()])

class DatasetType(Enum):
    cifar10 = 'cifar10'
    mnist = 'mnist'
    fashion_mnist = 'fashion_mnist'
    svhn = 'svhn'

### 三、模型训练

In [9]:
# 查看可用的训练设备
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [10]:
# 训练配置
batch_size = 128
epochs = 50
potential_dim = 8
dataset_type = DatasetType.mnist  # 在这里设置你的数据集

In [11]:
MODEL_PATH = f'models/cvae_{dataset_type.value}.pth'
HISTORY_PATH = f'history/cvae_{dataset_type.value}_history.npy'
CHANNELS = 1 if dataset_type in [DatasetType.mnist, DatasetType.fashion_mnist] else 3

# 模型和优化器
vae = ConditionalConvVAE(potential_dim=potential_dim, channels=CHANNELS)
vae.to(device)

running_losses = []
if os.path.exists(MODEL_PATH):
    try:
        vae.load_state_dict(torch.load(MODEL_PATH))
        running_losses = list(np.load(HISTORY_PATH)) if os.path.exists(HISTORY_PATH) else []
    except RuntimeError:
        pass

In [12]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

if dataset_type == DatasetType.mnist:
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
elif dataset_type == DatasetType.cifar10:
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
elif dataset_type == DatasetType.fashion_mnist:
    train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
elif dataset_type == DatasetType.svhn:
    train_dataset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
else:
    raise NotImplementedError

optimizer = optim.Adam(vae.parameters(), lr=0.001)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


p_bar = tqdm(range(epochs))
for epoch in p_bar:
    running_loss = np.array([0., 0., 0.])
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = vae(data, labels)
        loss, losses = vae_loss(recon_batch, data, mu, log_var)
        loss.backward()
        
        optimizer.step()
        running_loss += 1/(batch_idx + 1) * (losses - running_loss)
        p_bar.set_postfix(progress=f'{(batch_idx+1)/len(train_loader)*100:.2f}%', totalLoss=f'{running_loss[0]:.3f}', MSELoss=f'{running_loss[1]:.3f}', KLDLoss=f'{running_loss[2]:.3f}')
        
    running_losses.append(running_loss)
    np.save(HISTORY_PATH, np.array(running_losses))
    torch.save(vae.state_dict(), MODEL_PATH)

  0%|          | 0/50 [00:00<?, ?it/s, KLDLoss=1166.958, MSELoss=1523.447, progress=2.56%, totalLoss=2690.405]

tensor([[ 0.6900,  0.1343, -0.2119,  ...,  0.5840, -1.0846, -1.1241],
        [-0.3916,  0.3265, -0.2579,  ..., -0.8676, -0.3159, -1.3930],
        [ 0.2934,  1.0896,  2.2991,  ...,  1.0957,  0.4492, -1.4079],
        ...,
        [ 0.5124,  0.5268, -0.3763,  ..., -0.0225,  0.0106, -1.1052],
        [ 0.2934,  1.0896,  2.2991,  ...,  1.0957,  0.4492, -1.4079],
        [ 0.5124,  0.5268, -0.3763,  ..., -0.0225,  0.0106, -1.1052]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[-0.5112,  0.2828, -1.0068,  ...,  0.2114, -0.5222, -0.3816],
        [-0.9997,  0.4582,  0.2789,  ...,  0.3395, -0.6903,  0.3455],
        [ 0.2944,  1.0906,  2.2981,  ...,  1.0967,  0.4482, -1.4089],
        ...,
        [ 0.6890,  0.1353, -0.2129,  ...,  0.5830, -1.0836, -1.1231],
        [ 0.2944,  1.0906,  2.2981,  ...,  1.0967,  0.4482, -1.4089],
        [ 0.6890,  0.1353, -0.2129,  ...,  0.5830, -1.0836, -1.1231]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[ 0.9689,  1.4733, -

  0%|          | 0/50 [00:00<?, ?it/s, KLDLoss=1174.703, MSELoss=1521.132, progress=5.54%, totalLoss=2695.835]

tensor([[ 0.6870,  0.1365, -0.2158,  ...,  0.5839, -1.0859, -1.1254],
        [-0.9953,  0.4572,  0.2770,  ...,  0.3399, -0.6940,  0.3439],
        [ 0.6870,  0.1365, -0.2158,  ...,  0.5839, -1.0859, -1.1254],
        ...,
        [-0.9953,  0.4572,  0.2770,  ...,  0.3399, -0.6940,  0.3439],
        [ 0.2954, -0.5780,  0.0478,  ...,  0.6984, -1.4237,  0.3503],
        [ 0.2959,  1.0906,  2.2980,  ...,  1.0991,  0.4462, -1.4121]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[-0.5136,  0.2841, -1.0094,  ...,  0.2107, -0.5279, -0.3794],
        [-0.3899,  0.3242, -0.2577,  ..., -0.8682, -0.3137, -1.3943],
        [ 0.5148,  0.5264, -0.3797,  ..., -0.0217,  0.0079, -1.0988],
        ...,
        [ 0.8395, -0.7254, -0.6327,  ...,  0.5298,  0.2522, -0.4445],
        [ 0.9695,  1.4755, -0.2629,  ..., -0.2572, -0.6133,  0.4683],
        [-0.9953,  0.4574,  0.2768,  ...,  0.3400, -0.6940,  0.3438]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[ 0.2962, -0.5789,  

  0%|          | 0/50 [00:00<?, ?it/s, KLDLoss=1182.697, MSELoss=1507.867, progress=8.96%, totalLoss=2690.564]

...

  0%|          | 0/50 [00:02<?, ?it/s, KLDLoss=1186.091, MSELoss=1510.353, progress=43.28%, totalLoss=2696.444]

tensor([[-0.3934,  0.3233, -0.2576,  ..., -0.8626, -0.3157, -1.3931],
        [-0.9965,  0.4517,  0.2750,  ...,  0.3388, -0.6901,  0.3429],
        [-0.3934,  0.3233, -0.2576,  ..., -0.8626, -0.3157, -1.3931],
        ...,
        [ 0.2951, -0.5774,  0.0459,  ...,  0.6974, -1.4213,  0.3534],
        [ 0.2951, -0.5774,  0.0459,  ...,  0.6974, -1.4213,  0.3534],
        [-0.4462,  0.5820, -0.4448,  ...,  1.8268,  0.8034, -0.1611]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[-0.4459,  0.5820, -0.4447,  ...,  1.8269,  0.8032, -0.1612],
        [ 0.9649,  1.4745, -0.2652,  ..., -0.2595, -0.6117,  0.4648],
        [ 0.8385, -0.7262, -0.6323,  ...,  0.5328,  0.2519, -0.4427],
        ...,
        [ 0.6866,  0.1316, -0.2154,  ...,  0.5830, -1.0824, -1.1211],
        [-0.3932,  0.3234, -0.2574,  ..., -0.8625, -0.3156, -1.3932],
        [-0.9960,  0.4515,  0.2748,  ...,  0.3388, -0.6901,  0.3427]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[ 0.2980,  1.0839,  

  0%|          | 0/50 [00:03<?, ?it/s, KLDLoss=1186.085, MSELoss=1509.482, progress=46.27%, totalLoss=2695.567]

tensor([[ 0.9643,  1.4759, -0.2622,  ..., -0.2619, -0.6123,  0.4663],
        [-0.3943,  0.3212, -0.2562,  ..., -0.8637, -0.3147, -1.3903],
        [ 0.6875,  0.1309, -0.2129,  ...,  0.5824, -1.0800, -1.1213],
        ...,
        [-0.9945,  0.4521,  0.2723,  ...,  0.3408, -0.6915,  0.3422],
        [ 0.8386, -0.7270, -0.6297,  ...,  0.5324,  0.2531, -0.4404],
        [-0.4461,  0.5805, -0.4442,  ...,  1.8279,  0.8044, -0.1596]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[-0.9946,  0.4522,  0.2722,  ...,  0.3409, -0.6916,  0.3424],
        [-0.9946,  0.4522,  0.2722,  ...,  0.3409, -0.6916,  0.3424],
        [ 0.2945, -0.5765,  0.0444,  ...,  0.6985, -1.4216,  0.3526],
        ...,
        [ 0.5208,  0.5255, -0.3760,  ..., -0.0202,  0.0119, -1.0995],
        [ 0.2958,  1.0880,  2.2964,  ...,  1.0988,  0.4444, -1.4063],
        [-0.4463,  0.5803, -0.4443,  ...,  1.8277,  0.8043, -0.1595]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([[ 0.2946, -0.5763,  

KeyboardInterrupt: 

### 四、绘制损失曲线

In [ ]:
plt.figure(figsize=(12, 5))
total_loss, bce_loss, kld_loss = np.array(running_losses).T
# 绘制损失
plt.subplot(1, 2, 1)
plt.plot(total_loss, label='Training Loss')
plt.plot(bce_loss, label='Training MSE Loss')
plt.plot(kld_loss, label='Training KLD Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

### 五、模型推理

In [ ]:
def generate(model, labels, device='cuda'):
    with torch.no_grad():
        num_samples = len(labels)
        z = torch.randn(num_samples, model.potential_dim).to(device)
        labels = torch.LongTensor(np.array(labels)).to(device)
        # 通过解码器生成图像
        generated_images = model.decode(z, labels)
    return generated_images

def plot(images):
    plt.figure(figsize=(8, 8))
    channels = images.size(1)
    for i, image in enumerate(images):
        plt.subplot(8, 8, i+1)
        img = image.squeeze().cpu().numpy()
        if channels == 3:
            img = np.transpose(img, (2, 1, 0))
            
        plt.imshow(img, cmap='gray' if channels == 1 else None)
        plt.axis('off')
    plt.show()

In [ ]:
images = generate(vae, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device)
plot(images)

In [ ]:
!python cvae_visualizer.py --mnist
# !python cvae_visualizer.py --cifar10
# !python cvae_visualizer.py --fashion_mnist
# !python cvae_visualizer.py --svhn